In [1]:
%%capture
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import numpy as np
import utils

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.__version__)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=6000)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

Num GPUs Available:  1
2.15.0
1 Physical GPUs, 1 Logical GPUs


In [3]:
#set the necessary seeds
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

## Import datasets

### Breast cancer dataset

In [4]:
train_pairs_breast, val_pairs_breast, test_pairs_breast = utils.split_dataset(utils.create_list())
print(f"Number of training pairs: {len(train_pairs_breast)}")
print(f"Number of val pairs: {len(val_pairs_breast)}")
print(f"Number of test_pairs pairs: {len(test_pairs_breast)}")

Number of training pairs: 388
Number of val pairs: 129
Number of test_pairs pairs: 130


In [5]:
train_dataset_breast = utils.create_dataset(train_pairs_breast, augment=True)
val_dataset_breast = utils.create_dataset(val_pairs_breast)
test_dataset_breast = utils.create_dataset(test_pairs_breast)

In [6]:
batch_size = 8

In [7]:
#batch the dataset and shuffle the training set

train_dataset_breast = train_dataset_breast.shuffle(buffer_size=len(train_pairs_breast)).batch(batch_size)
val_dataset_breast = val_dataset_breast.batch(batch_size)
test_dataset_breast = test_dataset_breast.batch(batch_size)

### Skin cancer dataset

In [ ]:
train_dataset_skin = utils.create_dataset(utils.create_list_skin("ISIC-2017_Training_Data", "ISIC-2017_Training_Part1_GroundTruth"), augment=True)
val_dataset_skin = utils.create_dataset(utils.create_list_skin("ISIC-2017_Validation_Data", "ISIC-2017_Validation_Part1_GroundTruth"))
test_dataset_skin = utils.create_dataset(utils.create_list_skin("ISIC-2017_Test_v2_Data", "ISIC-2017_Test_v2_Part1_GroundTruth"))

In [ ]:
train_dataset_skin = train_dataset_skin.shuffle(buffer_size=2000).batch(batch_size)
val_dataset_skin = val_dataset_skin.batch(batch_size)
test_dataset_skin = test_dataset_skin.batch(batch_size)

### Brain cancer dataset

In [ ]:
train_pairs_brain, val_pairs_brain, test_pairs_brain = utils.split_dataset(utils.create_list_brain())
print(f"Number of training pairs: {len(train_pairs_brain)}")
print(f"Number of val pairs: {len(val_pairs_brain)}")
print(f"Number of test_pairs pairs: {len(test_pairs_brain)}")

In [ ]:
train_dataset_brain = utils.create_dataset(train_pairs_brain, augment=True)
val_dataset_brain = utils.create_dataset(val_pairs_brain)
test_dataset_brain = utils.create_dataset(test_pairs_brain)

In [ ]:
train_dataset_brain = train_dataset_brain.shuffle(buffer_size=len(train_pairs_brain)).batch(batch_size)
val_dataset_brain = val_dataset_brain.batch(batch_size)
test_dataset_brain = test_dataset_brain.batch(batch_size)

## Architecture definition

In [8]:
from keras import layers

#batch normalization and activation
def batchnorm_relu(input):
    x = layers.BatchNormalization()(input)
    x = layers.Activation("relu")(x)
    return x

In [9]:
#residual block
def residual_block(input, filters, kernel_size=3, strides=1, initializer="he_normal"):

    regularizer = tf.keras.regularizers.L2(0.001)

    #first convolutional layer
    x = batchnorm_relu(input)
    x = layers.Conv2D(filters, kernel_size, strides=strides, padding="same", kernel_initializer=initializer, kernel_regularizer=regularizer)(x)

    #second convolutional layer
    x = batchnorm_relu(x)
    x = layers.Conv2D(filters, kernel_size, strides=1, padding="same", kernel_initializer=initializer)(x)

    #shortcut connection
    shortcut = layers.Conv2D(filters, kernel_size=1, strides=strides, padding="same", kernel_initializer=initializer, kernel_regularizer=regularizer)(input)

    output = layers.Add()([x, shortcut])
    return output

In [10]:
#upsampling
def decoder(input, skip, filters, kernel_size=3, initializer="he_normal"):
    #upsample the input
    x = layers.UpSampling2D((2, 2))(input)

    #concantenate the skip connection
    x = layers.concatenate([x, skip], axis=-1)

    #residual block
    x = residual_block(x, filters, kernel_size, strides=1, initializer = initializer)

    return x

In [11]:
def res_unet(input_size, filters=16):
    input = layers.Input(input_size)
    regularizer = tf.keras.regularizers.L2(0.001)

    #stem block
    x = layers.Conv2D(filters, 3, strides=1, padding="same", kernel_initializer="he_normal", kernel_regularizer=regularizer)(input)
    x = batchnorm_relu(x)
    x = layers.Conv2D(filters, 3, strides=1, padding="same", kernel_initializer="he_normal", kernel_regularizer=regularizer)(x)
    s = layers.Conv2D(filters, 1, strides=1, padding="same", kernel_initializer="he_normal", kernel_regularizer=regularizer)(x)
    skip_1 = layers.add([x, s])

    #encoder
    skip_2 = residual_block(skip_1, filters * 2, strides=2)
    skip_3 = residual_block(skip_2, filters * 4, strides=2)
    skip_4 = residual_block(skip_3, filters * 8, strides=2)
    # skip_5 = residual_block(skip_4, filters * 16, strides=2)

    skip_4 = layers.Conv2D(filters * 16, kernel_size=(3, 3), strides=1, padding="same", kernel_initializer="he_normal", kernel_regularizer=regularizer)(skip_4)
    skip_4 = batchnorm_relu(skip_4)
    skip_4 = layers.Conv2D(filters * 16, kernel_size=(3, 3), strides=1, padding="same", kernel_initializer="he_normal", kernel_regularizer=regularizer)(skip_4)
    skip_4 = batchnorm_relu(skip_4)

    #decoder
    # first_decoder = decoder(skip_5, skip_4, filters * 8, initializer="he_normal")
    second_decoder = decoder(skip_4, skip_3, filters * 4, initializer="he_normal")
    third_decoder = decoder(second_decoder, skip_2, filters * 2, initializer="he_normal")
    fourth_decoder = decoder(third_decoder, skip_1, filters, initializer="he_normal")

    output = layers.Conv2D(1, kernel_size=(1, 1), padding="same", activation="sigmoid")(fourth_decoder)

    res_unet = tf.keras.Model(inputs=input, outputs=output)

    return res_unet

In [12]:
tf.keras.utils.get_custom_objects()["dice_loss"] = utils.dice_loss
tf.keras.utils.get_custom_objects()["bce_dice_loss"] = utils.bce_dice_loss
tf.keras.utils.get_custom_objects()["tversky_loss"] = utils.tversky_loss
tf.keras.utils.get_custom_objects()["dice_coefficient"] = utils.dice_coefficient
tf.keras.utils.get_custom_objects()["iou"] = utils.iou

In [13]:
res_unet = res_unet(input_size=(256, 256, 3))

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
lr_adapter = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_dice_coefficient", mode = "max", factor=0.1, patience=7, min_lr=1e-6)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_dice_coefficient", mode= "max", patience=13, restore_best_weights=True)

res_unet.compile(optimizer=optimizer, loss=utils.bce_dice_loss, metrics=[utils.dice_coefficient, utils.iou, "accuracy"])

In [14]:
res_unet.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 256, 256, 16)         448       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 256, 256, 16)         64        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 256, 256, 16)         0         ['batch_normalization[0][0

In [15]:
history = res_unet.fit(train_dataset_breast, validation_data=val_dataset_breast, epochs=100, callbacks=[lr_adapter, early_stopping])

Epoch 1/100


I0000 00:00:1738856942.156127     291 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


49/49 [==============================] - 43s 366ms/step - loss: 3.8161 - dice_coefficient: 0.2180 - iou: 0.1237 - accuracy: 0.8200 - val_loss: 4.4843 - val_dice_coefficient: 0.1686 - val_iou: 0.0935 - val_accuracy: 0.3042 - lr: 1.0000e-04
Epoch 2/100
49/49 [==============================] - 7s 126ms/step - loss: 3.5339 - dice_coefficient: 0.2925 - iou: 0.1725 - accuracy: 0.8502 - val_loss: 3.7212 - val_dice_coefficient: 0.1743 - val_iou: 0.0965 - val_accuracy: 0.7609 - lr: 1.0000e-04
Epoch 3/100
49/49 [==============================] - 7s 125ms/step - loss: 3.4571 - dice_coefficient: 0.3128 - iou: 0.1868 - accuracy: 0.8635 - val_loss: 3.4980 - val_dice_coefficient: 0.2072 - val_iou: 0.1170 - val_accuracy: 0.8698 - lr: 1.0000e-04
Epoch 4/100
49/49 [==============================] - 8s 127ms/step - loss: 3.3903 - dice_coefficient: 0.3285 - iou: 0.1979 - accuracy: 0.8663 - val_loss: 3.5262 - val_dice_coefficient: 0.2101 - val_iou: 0.1189 - val_accuracy: 0.8163 - lr: 1.0000e-04
Epoch 5/100